In [37]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [38]:
import prompts as p
from langchain.prompts import PromptTemplate


PRODUCTS_AMAZON = '../../datasets/Amazon.csv'

PRODUCTS_GOOGLE = '../../datasets/GoogleProducts.csv'

PRODUCT_MAPPING = '../../datasets/GoogleAmazonSampled_400.csv'

SAVED_PROMPTS_PATH = '../../evals/saved_prompts_GA.csv'

# Load data
df_A = pd.read_csv(PRODUCTS_AMAZON, encoding='latin-1')
df_B = pd.read_csv(PRODUCTS_GOOGLE, encoding='latin-1')
df_mapping = pd.read_csv(PRODUCT_MAPPING)


df_A.fillna('', inplace=True)
df_B.fillna('', inplace=True)
df_mapping.fillna('', inplace=True)

prompt_name = 'prompt1_en_short_temp_0_gpt3.5-0301_with_brand'
prompt = p.prompt1_en_short

MODEL = "gpt-3.5-turbo-0301"

MAX_TOKENS = 15


In [39]:
# generate column with prompt_name if not exists
if 'prompt_name' not in df_mapping.columns:
    df_mapping[prompt_name] = ""

In [40]:
df_mapping.head(30)

,Unnamed: 0,ltable_id,rtable_id,label,prompt1_en_short_temp_0_gpt3.5-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301,prompt1_en_short_temp_0_gpt3.5-0301_with_brand
0,0,363,2854,True,True,True,
1,1,1269,1433,True,True,True,
2,2,63,2129,True,True,False,
3,3,540,5,True,False,True,
4,4,1262,192,True,True,True,
5,5,1320,685,True,True,False,
6,6,1092,978,True,True,True,
7,7,1214,549,True,True,True,
8,8,1026,2327,True,True,False,
9,9,594,606,True,True,True,


In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI


In [42]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [43]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [44]:
def cut_length(string ,length = 250):
    return string[0:length]

In [45]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [46]:
saved_prompts_df = pd.read_csv(SAVED_PROMPTS_PATH)

def saveResponse(prompt, response, isMatch, index):
    # create new column if not exists 
    if "prompt_" + prompt_name not in saved_prompts_df.columns:
        saved_prompts_df["prompt_" + prompt_name] = ""
        saved_prompts_df["response_" + prompt_name] = ""
        saved_prompts_df["isMatch_" + prompt_name] = ""


    
    # save response prompt and isMatch to df at index
    saved_prompts_df.at[index, "prompt_" + prompt_name] = prompt
    saved_prompts_df.at[index, "response_" + prompt_name] = response
    saved_prompts_df.at[index, "isMatch_" + prompt_name] = isMatch
    
    

In [47]:
def evaluate(df, prompt, temp=0, start = 0):
    # start = len(df) - start - 1
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
        print("Using OpenAI")
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=temp,max_tokens=MAX_TOKENS)
    for i in range(start, len(df)):
        print(i)
        # Creating Prompt and sending request and printing informations
        brandl = ""
        brandr = ""
        if (df_A["manufacturer"][df["ltable_id"][i]]):
            brandl = "\nBrand: " + str(df_A["manufacturer"][df["ltable_id"][i]])
        if (df_A["manufacturer"][df["rtable_id"][i]]):
            brandr = "\nBrand: " + str(df_A["brmanufacturerand"][df["rtable_id"][i]])

        product1 = "Title: " + cut_length(str(df_A["title"][df["ltable_id"][i]] or "none")) + brandl + "\nDescription: " + cut_length(str(df_A["description"][df["ltable_id"][i]] or "none"))
        product2 = "Title: " + cut_length(str(df_B["title"][df["rtable_id"][i]] or "none")) + brandr + "\nDescription: " + cut_length(str(df_B["description"][df["rtable_id"][i]] or "none"))
        
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        
        try:
            response = chain.run(p1=product1, p2=product2, callbacks=[handler])        
        except Exception as e:
            print(e)
            i -= 1
            continue

        formatted_prompt = chain.prompt.format_prompt(p1=product1, p2=product2).text
        if(i < 5):
            print(formatted_prompt)
            print("__\n__")
            print(response)
            print("__\n__")
        
        #Processing Response    
        isMatch = False
        if ("True" in response or "true" in response or "yes" in response):
            isMatch = True
        elif ("False" in response or "false" in response or "no" in response):
            isMatch = False
        
        df[prompt_name][i] = isMatch


        saveResponse(formatted_prompt, response, isMatch, i)

In [48]:
evaluate(df_mapping, prompt,temp=0, start=0)


0


KeyError: 'brand'

In [ ]:
df_mapping


,Unnamed: 0,ltable_id,rtable_id,label,prompt1_en_short_temp_0_gpt3.5-0301,match_prompt_manual_fewshot_1_eval2_temp_0_gpt3.5_0301
0,0,363,2854,True,True,True
1,1,1269,1433,True,True,True
2,2,63,2129,True,True,False
3,3,540,5,True,False,True
4,4,1262,192,True,True,True
...,...,...,...,...,...,...
395,395,517,465,False,False,False
396,396,517,750,False,False,False
397,397,517,2839,False,False,False
398,398,517,984,False,False,False


In [ ]:
# saved_prompts_df

In [ ]:
# write df to csv
df_mapping.to_csv(PRODUCT_MAPPING, index=False)
saved_prompts_df.to_csv(SAVED_PROMPTS_PATH, index=False)
